In [1]:
import wandb

In [2]:
def download_distilbert():
    run = wandb.init()
    print("API KEY: 501ec742d9174ae2b5538dbba9d348d69f98ec93")
    artifact = run.use_artifact('splenderai/<pii_detection>/model-1rbdb33p:v2', type='model')
    artifact_dir = artifact.download()

In [3]:
download_distilbert()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /Users/alessandropranzo/.netrc


API KEY: 501ec742d9174ae2b5538dbba9d348d69f98ec93


wandb: Downloading large artifact model-1rbdb33p:v2, 254.21MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:1:4.2


In [4]:
def download_distilbert2():
    run = wandb.init()
    print("API KEY: 501ec742d9174ae2b5538dbba9d348d69f98ec93")
    artifact = run.use_artifact('splenderai/<pii_detection>/model-vv5zcoip:v1', type='model')
    artifact_dir = artifact.download()


In [5]:
download_distilbert2()

API KEY: 501ec742d9174ae2b5538dbba9d348d69f98ec93


wandb: Downloading large artifact model-vv5zcoip:v1, 254.21MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:0:54.2


In [6]:
import torch
from torch import nn
from aux import ensembler

In [8]:
class KingBert(nn.Module):
    def __init__(self, distilbert_tuned, albert_tuned):
        super.__init__()
        self.distilbert = distilbert_tuned
        self.albert = albert_tuned

        for distilbert_param in self.distilbert.parameters():
            distilbert_param.requires_grad = False

        for albert_param in self.albert.parameters():
            albert_param.requires_grad = False 
        
        #Here we have an alpha for each label
        self.alpha = nn.Parameter(0.5 * torch.ones(47), requires_grad = True)
        self.softmax = nn.Softmax(dim = 1)

    def forward(self, distilbert_input_ids, albert_input_ids, distil_attention_mask, alb_attention_mask, distilbert_word_ids, albert_word_ids):
        distilbert_output = self.distilbert(input_ids=distilbert_input_ids, attention_mask=distil_attention_mask)
        albert_output = self.albert(input_ids=albert_input_ids, attention_mask=alb_attention_mask)

        distilbert_output = self.softmax(distilbert_output.logits)
        albert_output = self.softmax(albert_output.logits)

        distilbert_fixed, albert_fixed = ensembler(distilbert_output, albert_output, distilbert_word_ids, albert_word_ids)

        final_output = distilbert_fixed * self.alpha + albert_fixed * (torch.ones(47) - self.alpha)

        return final_output